[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/multimodal.ipynb)

# Multi-modal

Here, we'll showcase how to curate and register ECCITE-seq data from [Papalexi21](https://www.nature.com/articles/s41592-019-0392-0) in the form of [MuData](https://github.com/scverse/mudata) objects.

ECCITE-seq is designed to enable interrogation of single-cell transcriptomes together with surface protein markers in the context of CRISPR screens.

[MuData objects](https://mudata.readthedocs.io) build on top of AnnData objects to store multimodal data.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install 'lamindb[jupyter,bionty]'
!lamin init --storage ./test-multimodal --modules bionty

In [ ]:
import lamindb as ln
import bionty as bt

In [ ]:
mdata = ln.core.datasets.mudata_papalexi21_subset()
mdata

## Validate annotations 

In [ ]:
curate = ln.Curator.from_mudata(
    mdata,
    var_index={
        "rna": bt.Gene.symbol,  # gene expression
        "adt": bt.CellMarker.name,  # antibody derived tags reflecting surface proteins
        "hto": ln.Feature.name,  # cell hashing
        "gdo": ln.Feature.name,  # guide RNAs
    },
    categoricals={
        "perturbation": ln.ULabel.name,  # shared categorical
        "replicate": ln.ULabel.name,  # shared categorical
        "hto:technique": bt.ExperimentalFactor.name,  # note this is a modality specific categorical
    },
    organism="human",
)

In [ ]:
# optional: register additional columns we'd like to curate
curate.add_new_from_columns(modality="rna")
curate.add_new_from_columns(modality="adt")
curate.add_new_from_columns(modality="hto")
curate.add_new_from_columns(modality="gdo")

In [ ]:
curate.validate()

In [ ]:
# add new var index
curate.add_new_from_var_index("rna")
curate.add_new_from_var_index("hto")
curate.add_new_from_var_index("gdo")

# add new categories
curate.add_new_from("perturbation")
curate.add_new_from("replicate")

In [ ]:
curate.validate()

## Register curated artifact

In [ ]:
artifact = curate.save_artifact(description="Sub-sampled MuData from Papalexi21")

In [ ]:
artifact.describe()

In [ ]:
# clean up test instance
!rm -r test-multimodal
!lamin delete --force test-multimodal